<h1><center>Machine Learning-basierte Extraktion von phänotypischen Kennzahlen aus UAV-Bildern</center></h1>


<h3><center> height_estimator.ipynb: Höhe der Pflanze </center></h3>


Dieses Notebook beschreibt die Schritte zur Schätzung der Höhe einer Pflanze auf der Grundlage von Bildern, von der Datenorganisation bis zum Trainingsprozess des Regressionsmodells.

## Bibliotheken

In [1]:
# Standard Libraries
import os
import glob as gb
import random

# External Libraries
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image, UnidentifiedImageError

# Data Science and Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.base import np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Image Processing Libraries
import skimage.feature
#from skimage.feature.texture import greycomatrix, greycoprops

# Visualization and Display
from IPython.display import Image, display
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

# Deep Learning Libraries
import torch
import torchvision
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from torchvision.transforms import v2 as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

# TensorFlow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

import utils
device = torch.device('cuda')

/tmp/ipykernel_18994/1782121719.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_18994/1782121719.py:30: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')
2024-03-07 14:47:15.068189: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024

## Datenvorbereitung

### Extraktion von Pflanzen-IDs

In [2]:
def create_dataframe(base_dir, sub_dirs):
    data = {'plant_id': [], 'height': []}
    
    for dir_name in sub_dirs:
        dir_path = os.path.join(base_dir, dir_name)
        
        for filename in os.listdir(dir_path):
            if filename.endswith('.jpg'):
                plant_id = os.path.splitext(filename)[0]  # Remove the .tif extension
                data['plant_id'].append(plant_id)
                data['height'].append(0)  # Height is blank for now

    df = pd.DataFrame(data)
    return df

base_dir = 'raid/GrowliFlowerO/Field1/'
sub_dirs = ['2020_09_08', '2020_11_02', '2020_09_22', '2020_08_25', '2020_10_19', '2020_08_12', '2020_08_19', '2020_10_27', '2020_10_06', '2020_10_29', '2020_09_17', '2020_09_02']

df = create_dataframe(base_dir, sub_dirs)
#df.to_csv('raid/harvested_plants.csv')

### Extraktion von Pflanzenhöhen

In [3]:
def extract_heights(base_dir, df_path):
    df = pd.read_csv(df_path)
    
    for index, row in df.iterrows():
        plant_id = row['plant_id']
        parts = plant_id.split('_')
        date = '_'.join(parts[0:3])
        plot = parts[4][:5]  
        row = parts[5][0]
        position = parts[5][1:]
        # Construct the path to the annotation file
        annotation_path = os.path.join(base_dir, date, plot, f'{date}_{plot}.csv')
        # Read the annotation file
        annotation_df = pd.read_csv(annotation_path)
        # Extract the height
        height = annotation_df.loc[(annotation_df['Row'] == row) & (annotation_df['Position'] == int(position)), 'Height'].values[0]
        # Update the height in the main dataframe
        df.loc[index, 'height'] = height
    return df

base_dir = 'raid/Annotations/Field1/'
df_path = 'raid/field1_plants_heights.csv'

df = extract_heights(base_dir, df_path)
#df.to_csv(df_path, index=False)

#### Nullwerte entfernen

In [4]:
nan_height = df[df['height'].isna()]
len(nan_height)
#df = df.dropna(subset=['height'])

0

Die organisierten Daten werden in einer csv-Datei gespeichert

#### Berechnung des Höhenbereichs der Pflanzen

In [5]:
height_range = df['height'].max() - df['height'].min()
print('Range of height:', height_range)

Range of height: 85.0


#### Laden der Anmerkungsdatei (Trainingsdatensatz)

In [7]:
annotations = pd.read_csv('raid/field1_plants_heights.csv')
annotations.head()

,plant_id,height
0,2020_09_08_Ref_Plot2_A4,39.0
1,2020_09_08_Ref_Plot4_A99,21.0
2,2020_09_08_Ref_Plot4_B98,33.0
3,2020_09_08_Ref_Plot1_B96,33.0
4,2020_09_08_Ref_Plot3_B7,41.0


## Training des Regressionmodell

### Vorbereitung der Daten (Features - Tragets)

Tragets:

In [8]:
numerical_targets = []
image_paths = []
for plant_id in annotations['plant_id']:
    parts = plant_id.split('_')
    date = '_'.join(parts[0:3])
    image_path = 'raid/GrowliFlowerO/Field1/' + date + '/' + plant_id + '.jpg'
    image_paths.append(image_path)
    numerical_targets.append(annotations['height'])
numerical_targets = numerical_targets[0]

Features:

### Merkmale Extraktion (nimmt so viel Zeit in Anspruch) 

In [16]:
# Funktion zum Extrahieren von Merkmalen der Bildern mit dem VGG16-Modell.
def VGG16_feature_extratctor(image_path):
# Laden des VGG16-Modells mit vortrainierten Gewichten von ImageNet ohne die obersten Schichten (include_top=False).
    model = VGG16(weights='imagenet', include_top=False)
    img = kimage.load_img(image_path, target_size=(490, 490))
    x = kimage.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    VGG16_features = model.predict(x)
    return VGG16_features

In [20]:
# Extract features for all images
features = np.array([VGG16_feature_extratctor(path) for path in image_paths])
features = features.reshape(features.shape[0], -1)  # Flatten the features

2023-12-28 20:53:11.540454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21372 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5
2023-12-28 20:53:12.420602: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-28 20:53:12.518297: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 102ms/step


Da der Prozess der Merkmalsextraktion Zeit in Anspruch nimmt, werden die Merkmale gespeichert und geladen, um Zeit zu sparen.

In [27]:
# Speichern der Merkmale in einer Datei
np.save('raid/features.npy', features)

In [9]:
features = np.load('raid/features.npy')
targets = np.array(numerical_targets) 

###  Aufbauen des Regressionsmodells

In [10]:
def build_regression_model(input_shape):
    model = Sequential()
    
    # Eingabeschicht
    model.add(Dense(512, activation='relu', input_dim=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Versteckte Schicht 1
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    # Versteckte Schicht 2
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Versteckte Schicht 3
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Versteckte Schicht 4
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    # Versteckte Schicht 5
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Versteckte Schicht 6
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Ausgabeschicht für Regression
    model.add(Dense(1))

    # Kompilieren des Modells mit dem Adam-Optimierer und der mittleren quadratischen Fehlerverlustfunktion für Regression
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

### Modell Training

In [10]:
X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.2)

# Erstellen des Modells
regression_model = build_regression_model(X_train.shape[1])

# Trainieren des Modells mit den Trainingsdaten.
history = regression_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64)

2024-03-07 14:13:07.963692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18777 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5


### Bewertung des Modells

Dieser Codeabschnitt führt die Bewertung des Regressionsmodells durch, indem er verschiedene Metriken berechnet und ausgibt, die Aufschluss über die Genauigkeit und Effektivität des Modells geben.

In [11]:
from keras.models import load_model

regression_model = load_model('raid/height_regression_model.h5')

2024-03-07 14:50:36.742998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18777 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5


In [13]:
X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.2)
val_predictions = regression_model.predict(X_val)
mse = mean_squared_error(y_val, val_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, val_predictions)
r2 = r2_score(y_val, val_predictions)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

11/11 [==============================] - 0s 8ms/step


2024-03-07 14:51:19.304339: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Mean Squared Error: 17.694231109222052
Root Mean Squared Error: 4.20645113001709
Mean Absolute Error: 2.2479528723341047
R-squared: 0.9597292293634427


## Inferenz

In [14]:
def predict_numerical_value(image_path, regression_model):
    new_features = VGG16_feature_extratctor(image_path).reshape(1, -1)
    predicted_value = regression_model.predict(new_features)
    return predicted_value

In [17]:
image_path = 'raid/GrowliFlowerO/Field2/2021_09_03/2021_09_03_Ref_Plot1_A19.jpg'
predicted_value = predict_numerical_value(image_path, regression_model)
print(predicted_value) 

2024-03-07 14:51:36.849274: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-07 14:51:36.946265: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 27ms/step
[[70.80095]]


: 